In [1]:
import sys
sys.executable

'/opt/anaconda3/envs/resume_env/bin/python'

In [5]:
! pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]


In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)

data = pd.read_excel('./BR_RawData.xlsx')

data.head()

,Auto req ID,Current Req Status,Grade,Designation,Recruiter,Department Type,BU,Client Interview?,Mandatory Skills,Entity,Client Name,Billing Type,Project,Requester ID,TAG Manager,RM Name,Job description,Joining Location,Backfill for Employee Name,Date Approved,No. of Positions,Positions Remaining,Sourcing Type,Requirement Type,ST (Bill Rate) Enter only numeric value and 0 for Non-Billable
0,26640BR,Open,E3,Senior Software Engineer,Shubhi Priya (1027585),Technical,ITS - TMH - Delivery,Yes,"CSS,Hibernate,HTML,Java,JavaScript,React JS,Re...",OFFSHORE,IRON MOUNTAIN,Billable,IM TM 2025,1017237,"Antony, Nithin (1027544)","Hippargi, Anil (1017237)",Java Full Stack Developer-Sr. Tech LeadSummary...,Bangalore - Campus,NaN,2024-11-28,1,1,External - India,New,28.0
1,31840BR,Open,E3,Senior Network Engineer,NaN,Technical,ITS - TMH - Delivery,Yes,Cisco Certified Network Associate (CCNA),OFFSHORE,Yahoo Holdings Inc,Billable,Yahoo Application Support,1015402,"Antony, Nithin (1027544)","Sinai Kakodkar, Bhasker (1015402)",Network SecurityInternet protocolsJunipe/Cisco,Gurgaon,NaN,2025-04-17,1,1,External - India,New,32.0
2,32228BR,Open,NaN,NaN,Jagadeesh Murugan (1038064),Technical,ITS - TMH - Delivery,Yes,"AWS,Java","Infinite Computer Solutions, Inc.",VERIZON Corp Serv Group Inc,Billable,Vz-Value BRM - Acc Balance- Services,20117703,"Jain, Rishabh (2009983)","Kaparthi, Aravind (20109477)","Technical Skills:Java, J2EE, Spring, Spring bo...",Texas,NaN,2025-04-27,1,1,External - US,New,115.0
3,32663BR,Open,E1,Associate Support Engineer,Shyam Sheriel (1030409),Support,ITS - TMH - Delivery,Yes,"ITIL Foundation,Monitoring",OFFSHORE,IMS Convergence,Billable,IMS - Convergence,1026374,"Antony, Nithin (1027544)","Thampy, Sunil (1033691)",L1 Support EngineerInfinite Convergence Soluti...,Bangalore - Global Axis,NaN,2025-06-14,1,1,External - India,New,5.5
4,32665BR,Approved,E2,Support Engineer,Manjula Venkatesh (1034557),Support,ITS - TMH - Delivery,Yes,"ITIL Foundation,Monitoring",OFFSHORE,IMS Convergence,Billable,IMS - Convergence,1026374,"Antony, Nithin (1027544)","Thampy, Sunil (1033691)",L1 Support EngineerInfinite Convergence Soluti...,Bangalore - Global Axis,NaN,2025-08-19,1,1,External - India,New,5.5


In [93]:
df = data[['Auto req ID', 'Job description']].rename(columns={
    'Auto req ID': 'business_req_id', 
    'Job description': 'job_description'})

In [94]:
pd.set_option("display.max_colwidth", None)
df.head()

,business_req_id,job_description
0,26640BR,"Java Full Stack Developer-Sr. Tech LeadSummary:The Sr. Tech Lead will provide design and delivery of a portal web application within the agile development lifecycle. The ideal candidate will be an experienced engineer with web application development with micro services and REST APIs in highly performant systems. The candidate has at least 8 years of experience developing a web application. To be successful, the candidate needs to show experience with Java, springboot, Microservices and Javascript Frameworks, e.g., Angular, REACT etc. and REST APIs. The candidate should also have Cloud-based development experience.Responsibilities:Design and architect Java-based applications and solutions to meet business requirementsLead and mentor development teams in best practices and architectural guidelinesCollaborate with stakeholders to gather and analyze requirements, and propose technical solutionsDevelop high-quality, scalable, and maintainable code in JavaIdentify technical debt and propose strategies for refactoring and optimizationStay current with emerging technologies and industry trends, and incorporate them into architectural designsParticipate in design discussions about the technical implementation and consider the tradeoffs to support business value, scalability and delivery timeline.Develop web UI pages using JavaScript based frameworks, e.g., Angular, REACT, Nuxeo etcWork with the services team to integrate with service REST APIs, JSON etc.Develop back-end components and services to improve responsiveness and overall performanceWrite effective, scalable codeIntegration with Client Facing Application and UpStream APDevelop unit tests with a test driven mindsetDebug and address performance issuesDesign and code for security considerations, anticipates security risksPeer / code reviewsLead the development of a product featureQualifications:Strong communication, collaboration and problem solving skills with a track record of delivering production grade systems in a team environmentMotivated individual who learns quickly, has pride in building a new product and can engage others to accelerate technical solutionsFamiliarity with implementation design patterns and performance challengesMinimum Bachelor’s degreeExperience in working with distributed teamsAt least 7 years of experience working with agile scrum methodologiesAt least 3+ years of Javascript, HTML5, CSS3, REACT etc.Proven 7+ years of experience as a Java Developer including backend and middle tier development based on Microservices PattersProficient in the following technologies/LanguagesJava,Spring FrameworkSpring BootSQLAPI Integration (REST, SOAP)JSON and XMLGood understanding of enterprise solutions including storage,, and software layers.Hands on Experience with implementation of Cross Cutting Concern including Caching, Logging and Tracing.Working Knowledge and Understanding of Microservices architectureUnderstanding of Docker and Kubernetes and KubeflowUnderstanding of data base like SQL (Oracle, MS-SQL) and non-SQL (Mongo) is must.Proven internal/external written and verbal communication, interpersonal and analytical skillsAbility to multitask and prioritize issues in a complex environment.Understanding of Machine Learning and AI techniques is a plus.Understanding of Cloud Technologies and solutions - GCP Preferred is a plusStrong exposure to Application integration with AI ML, DB such as Mongo DB and Cloud exposure (AWS and GDS)Enterprise Architecture exposureExperience and Expertise in building and leading technical teamsStrong CommunicationHands on, willing to do R&D and eye for detailsExperience with agile tools such as Atlassian JIRA, Rally, TFS or Version OneMastery of code repositories such as subversion / git and artifactory reposExperience with DockerExperience with web server, e.g., nginx, Tomcat is a plus"
1,31840BR,Network SecurityInternet protocolsJunipe/Cisco
2,32228BR,"Technical Skills:Java, J2EE, Spri

In [95]:
df.shape

(275, 2)

In [96]:
from openai import OpenAI
import json
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI()



In [97]:
import re

def extract_skills_and_experience(jd_text: str):
    """
    Extract required skills and experience from a Job Description 
    using GPT-4o-mini and return structured JSON.
    """

    prompt = f"""
    Extract the REQUIRED SKILLS and EXPERIENCE from the following Job Description.
    Return ONLY valid JSON with two keys: "skills" (list of strings) and
    "experience" (list of strings).

    Job Description:
    {jd_text}
    """

    response = client.responses.create(
        model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
        input=prompt,
        max_output_tokens=400
    )

    # Extract JSON text from the model output
    json_text = response.output_text

    pattern = r"```json\s*(\{[\s\S]*?\})\s*```"
    match = re.search(pattern, json_text)

    if match:
        json_text = match.group(1)
        data = json.loads(json_text)
    else:
        data = None
    
    return data

In [6]:
df.shape

(275, 2)

In [98]:
df1 = df[:50]
df1.shape

(50, 2)

In [8]:
sample_jd = df1.iloc[1]['job_description']
sample_jd

'Network SecurityInternet protocolsJunipe/Cisco'

In [19]:
result = extract_skills_and_experience(sample_jd)
result

{'skills': ['Network Security', 'Internet protocols', 'Juniper', 'Cisco'],
 'experience': []}

In [20]:
type(result)

dict

In [16]:
result['skills']

['Network Security', 'Internet protocols', 'Juniper', 'Cisco']

In [100]:
import pandas as pd

br_skills = pd.DataFrame(columns=['brno', 'skills', 'experience'])

for index, row in df1.iterrows():
    jd = row['job_description']
    # print(f'job description: {jd}\n')
    result = extract_skills_and_experience(jd)
    # print(f"Business Req ID: {row['business_req_id']}")
    # print(json.dumps(result, indent=2))
    # print("\n" + "="*50 + "\n")
    try:
        skills = ', '.join(result['skills'])
        experience = ', '.join(result['experience'])
    except Exception as e:
        print(f'Error processing BR ID {row["business_req_id"]}: {str(e)}')
        continue

    br_skills.loc[len(br_skills)] = [row['business_req_id'], skills, experience]

Error processing BR ID 36612BR: 'NoneType' object is not subscriptable


In [102]:
br_skills.shape
br_skills.to_csv('br_skills.csv', index=False)

In [103]:
!ls -lh .

total 3400
-rw-r--r--@   1 mansoor  staff   1.0M Nov 21 11:50 01. Resume Selector Design Document.docx
-rw-r--r--@   1 mansoor  staff   220K Nov 21 12:10 BR_RawData.xlsx
-rw-r--r--@   1 mansoor  staff   217K Nov 21 11:48 Offshore_Open_BR_22Oct_V1.xlsx
-rw-r--r--@   1 mansoor  staff    82K Nov 21 11:48 TMH_Pool_Active_22Oct_BR MAPPING.xlsx
-rw-r--r--@   1 mansoor  staff    21K Nov 21 19:24 br_skills.csv
-rw-r--r--@   1 mansoor  staff    63K Nov 21 19:14 candidate_skills.csv
drwxr-xr-x@ 122 mansoor  staff   3.8K Nov 21 16:41 it_resumes
-rw-r--r--@   1 mansoor  staff     0B Nov 21 12:12 jd_skill_gen.py
-rw-r--r--@   1 mansoor  staff    63B Nov 21 16:45 requirements.txt
-rw-r--r--@   1 mansoor  staff    65K Nov 21 19:24 test.ipynb
-rw-r--r--@   1 mansoor  staff     0B Nov 21 12:12 test.py


In [ ]:
# https://huggingface.co/datasets/opensporks/resumes/tree/main/data/data/INFORMATION-TECHNOLOGY

from huggingface_hub import snapshot_download

repo_id = "opensporks/resumes"
subdir = "data/data/INFORMATION-TECHNOLOGY"

local_dir = snapshot_download(
    repo_id=repo_id,
    repo_type="dataset",
    revision="main",  # or any specific version / branch
    local_dir="resumes_it",  # where to save locally
    local_dir_use_symlinks=False,
    allow_patterns=[f"{subdir}/*"]
)

print("Downloaded to:", local_dir)


/opt/anaconda3/envs/resume_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/resume_env/lib/python3.12/site-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `snapshot_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 120 files: 100%|██████████| 120/120 [00:10<00:00, 11.40it/s]

Downloaded to: /Users/mansoor/Documents/genai/05_resume_selector/resumes_it


In [50]:
!mkdir it_resumes
!mv resumes_it/data/data/INFORMATION-TECHNOLOGY/* it_resumes/

In [51]:
!rm -rf resumes_it

In [84]:
import pdfplumber
from openai import OpenAI
import json
import re
from pydantic import BaseModel
from typing import List

# ---------------------------
# Pydantic Schema
# ---------------------------
class ResumeSchema(BaseModel):
    skills: List[str]
    experience: List[str]

# ---------------------------
# Initialize OpenAI Client
# ---------------------------
client = OpenAI()  # Make sure OPENAI_API_KEY is set in your environment

# ---------------------------
# Helper function to clean LLM JSON
# ---------------------------
def clean_llm_json(llm_output: str) -> str:
    """
    Extract JSON from LLM output that may contain Markdown or extra text.
    """
    pattern = r"```json\s*(\{[\s\S]*?\})\s*```"
    match = re.search(pattern, llm_output)
    if match:
        return match.group(1)
    # fallback: extract first { ... } block
    start = llm_output.find("{")
    end = llm_output.rfind("}") + 1
    if start != -1 and end != -1:
        return llm_output[start:end]
    raise ValueError("No JSON found in LLM output")

# ---------------------------
# Main function
# ---------------------------
def extract_skills_from_pdf(pdf_path: str) -> ResumeSchema:
    """
    Extract skills and experience from a resume PDF using GPT-4o-mini and return validated JSON.
    """
    # 1. Extract text from PDF
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"

    if not text.strip():
        raise ValueError("No text could be extracted from the PDF.")

    # 2. Prepare resume-selector prompt
    prompt = f"""
You are an intelligent assistant designed to extract structured information from candidate resumes
for a resume selection system.

Instructions:
- Return ONLY valid JSON, no explanations, no backticks, no extra text.
- Use EXACTLY the keys: "skills" and "experience".
- "skills": list of strings of all relevant technical and professional skills.
- "experience": list of strings describing work experience and achievements.
- Include hard and relevant soft skills. Include detailed experience statements.
- If data is missing, return an empty list, never omit keys.

JSON Example:
{{
  "skills": ["Java", "Spring Boot", "Microservices", "AWS"],
  "experience": [
    "Developed a microservices-based system using Java and Spring Boot.",
    "Led a team of 5 engineers in agile project delivery."
  ]
}}

Resume Text:
{text}
"""

    # 3. Call LLM
    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt,
        max_output_tokens=500
    )

    # 4. Clean JSON and validate
    try:
        json_text = clean_llm_json(response.output_text)
        parsed_json = json.loads(json_text)
        resume_data = ResumeSchema(**parsed_json)
    except Exception as e:
        print(f'{pdf_path=}')
        print(f'{json_text=}')
        print(f'Encountered an exception: {str(e)}')
        resume_data = {}

    return resume_data

In [86]:
from glob import glob
from pathlib import Path

candidate_skills = pd.DataFrame(columns=['emp_id', 'skills', 'experience'])

pdf_files = glob("./it_resumes/*.pdf")
for pdf_file in pdf_files[:50]:

    p = Path(pdf_file)
    try:
        result = extract_skills_from_pdf(pdf_file)
        skills = ', '.join(result.skills)
        experience = ', '.join(result.experience)
    except Exception as e:
        print(f'skipping the resume: {p.stem}')
        continue

    candidate_skills.loc[len(candidate_skills)] = [p.stem, skills, experience]


pdf_path='./it_resumes/38753827.pdf'
json_text=''
Encountered an exception: Expecting value: line 1 column 1 (char 0)
skipping the resume: 38753827
pdf_path='./it_resumes/27372171.pdf'
json_text=''
Encountered an exception: Expecting value: line 1 column 1 (char 0)
skipping the resume: 27372171
pdf_path='./it_resumes/41344156.pdf'
json_text=''
Encountered an exception: Expecting value: line 1 column 1 (char 0)
skipping the resume: 41344156
pdf_path='./it_resumes/20674668.pdf'
json_text=''
Encountered an exception: Expecting value: line 1 column 1 (char 0)
skipping the resume: 20674668
pdf_path='./it_resumes/83816738.pdf'
json_text=''
Encountered an exception: Expecting value: line 1 column 1 (char 0)
skipping the resume: 83816738
pdf_path='./it_resumes/27295996.pdf'
json_text=''
Encountered an exception: Expecting value: line 1 column 1 (char 0)
skipping the resume: 27295996


In [90]:
candidate_skills.shape, type(candidate_skills)

((44, 3), pandas.core.frame.DataFrame)

In [92]:
candidate_skills.to_csv('candidate_skills.csv', index=False)